# 1. Needleman & Wunsch

In [99]:
### Question 1
import numpy as np

import numpy as np

def creer_matrice_distance(alphabet, score_match, score_mismatch):
    taille = len(alphabet)
    matrice_distance = np.zeros((taille, taille))
    
    for i in range(taille):
        for j in range(taille):
            if i == j:
                matrice_distance[i][j] = score_match  # Match
            else:
                matrice_distance[i][j] = score_mismatch  # Mismatch
    
    return matrice_distance

# exemple avec les nucléotides
alphabet = ['A', 'C', 'G', 'T']
score_match = 1
score_mismatch = -1

matrice_distance = creer_matrice_distance(alphabet, score_match, score_mismatch)
print("Matrice de distance :")
print(matrice_distance)

Matrice de distance :
[[ 1. -1. -1. -1.]
 [-1.  1. -1. -1.]
 [-1. -1.  1. -1.]
 [-1. -1. -1.  1.]]


In [100]:
#Question 2 :

def alignement_needleman_wunsch(seq1, seq2, matrice_distance, cout_gap):
    # determination de la longueur des séquences
    len_seq1 = len(seq1)
    len_seq2 = len(seq2)
    
    # on initialise la matrice de score
    matrice_score = np.zeros((len_seq1 + 1, len_seq2 + 1))

    # on remplit la première colonne et de la première ligne
    for i in range(1, len_seq1 + 1):
        matrice_score[i][0] = i * cout_gap  # = coût de gap
    for j in range(1, len_seq2 + 1):
        matrice_score[0][j] = j * cout_gap  # = coût de gap

    # on remplit la matrice de score selon la récurrence
    for i in range(1, len_seq1 + 1):
        for j in range(1, len_seq2 + 1):
            match_mismatch_score = matrice_distance[alphabet.index(seq1[i - 1])][alphabet.index(seq2[j - 1])]
            matrice_score[i][j] = max(
                matrice_score[i - 1][j - 1] + match_mismatch_score,  # match/mismatch
                matrice_score[i - 1][j] + cout_gap,                 # gap dans seq2
                matrice_score[i][j - 1] + cout_gap                  # gap dans seq1
            )

    return matrice_score


In [101]:
# test de la fonction
alphabet = ['A', 'C', 'G', 'T']
score_match = 1
score_mismatch = -1
cout_gap = -2

matrice_distance = creer_matrice_distance(alphabet, score_match, score_mismatch)

seq1 = "CAT"
seq2 = "TCT"


matrice_score = alignement_needleman_wunsch(seq1, seq2, matrice_distance, cout_gap)
print("Matrice de score :")
print(matrice_score)

Matrice de score :
[[ 0. -2. -4. -6.]
 [-2. -1. -1. -3.]
 [-4. -3. -2. -2.]
 [-6. -3. -4. -1.]]


In [102]:
#Question 3 : on définit la fonction Traceback

def traceback(matrice_score, seq1, seq2, matrice_distance, cout_gap):
    alignement_seq1 = ""
    alignement_seq2 = ""
    i, j = len(seq1), len(seq2)
    
    while i > 0 or j > 0:
        if i > 0 and j > 0 and matrice_score[i][j] == matrice_score[i - 1][j - 1] + matrice_distance[alphabet.index(seq1[i - 1])][alphabet.index(seq2[j - 1])]:
            alignement_seq1 += seq1[i - 1]
            alignement_seq2 += seq2[j - 1]
            i -= 1
            j -= 1
        elif i > 0 and matrice_score[i][j] == matrice_score[i - 1][j] + cout_gap:
            alignement_seq1 += seq1[i - 1]
            alignement_seq2 += '-'
            i -= 1
        else:
            alignement_seq1 += '-'
            alignement_seq2 += seq2[j - 1]
            j -= 1

    # pour prendre en compte le fait que l'on a construit la liste en partant de la fin des séquences, on les inverse
    alignement_seq1 = alignement_seq1[::-1]
    alignement_seq2 = alignement_seq2[::-1]

    return alignement_seq1, alignement_seq2


In [103]:
#Nouvelle fonction d'alignement (comprenant un appel de la fonction traceback) : 

def alignement_needleman_wunsch(seq1, seq2, matrice_distance, cout_gap):
    len_seq1 = len(seq1)
    len_seq2 = len(seq2)
    
    # on initialise la matrice
    matrice_score = np.zeros((len_seq1 + 1, len_seq2 + 1))

    # on remplit la première colonne et la première ligne suivant les paramètres choisis
    for i in range(1, len_seq1 + 1):
        matrice_score[i][0] = i * cout_gap 
    for j in range(1, len_seq2 + 1):
        matrice_score[0][j] = j * cout_gap  # coût des gap

    # on remplit le restant de la matrice de score selon la récurrence
    for i in range(1, len_seq1 + 1):
        for j in range(1, len_seq2 + 1):
            match_mismatch_score = matrice_distance[alphabet.index(seq1[i - 1])][alphabet.index(seq2[j - 1])]
            matrice_score[i][j] = max(
                matrice_score[i - 1][j - 1] + match_mismatch_score,  # match ou mismatch
                matrice_score[i - 1][j] + cout_gap,                 # pour un gap dans la seq2
                matrice_score[i][j - 1] + cout_gap                  # pour un gap dans la seq1
            )

    # appel de la fonction de traceback (définit ci-dessus)
    alignement_seq1, alignement_seq2 = traceback(matrice_score, seq1, seq2, matrice_distance, cout_gap)
    
    score_final = matrice_score[len_seq1][len_seq2]
    
    return alignement_seq1, alignement_seq2, score_final



In [104]:
# test
alphabet = ['A', 'C', 'G', 'T']
score_match = 1
score_mismatch = -1
cout_gap = -2

# on fait la matrice de distance
matrice_distance = creer_matrice_distance(alphabet, score_match, score_mismatch)

# on pose les séquences à aligner
seq1 = "CAT"
seq2 = "TCT"

# on procède au calcul de l'alignement
alignement_seq1, alignement_seq2, score_final = alignement_needleman_wunsch(seq1, seq2, matrice_distance, cout_gap)
print("Alignement Seq1 : ", alignement_seq1)
print("Alignement Seq2 : ", alignement_seq2)
print("Score final : ", score_final)


Alignement Seq1 :  CAT
Alignement Seq2 :  TCT
Score final :  -1.0


In [ ]:
#Question 4 : 

In [105]:
#Nouvelle fonction prenant en compte des couts de gaps différents suivant la nature du gap

def alignement_needleman_wunsch_gaps(seq1, seq2, matrice_distance, cout_gap_ouverture, cout_gap_prolongement):
    len_seq1 = len(seq1)
    len_seq2 = len(seq2)
    
    # on initialise la matrice de score
    matrice_score = np.zeros((len_seq1 + 1, len_seq2 + 1))

    # on remplot la première colonne et de la première ligne
    for i in range(1, len_seq1 + 1):
        matrice_score[i][0] = i * cout_gap_prolongement  # Coût de prolongement
    for j in range(1, len_seq2 + 1):
        matrice_score[0][j] = j * cout_gap_prolongement  # Coût de prolongement

    # on remplit la matrice de score
    for i in range(1, len_seq1 + 1):
        for j in range(1, len_seq2 + 1):
            match_mismatch_score = matrice_distance[alphabet.index(seq1[i - 1])][alphabet.index(seq2[j - 1])]
            matrice_score[i][j] = max(
                matrice_score[i - 1][j - 1] + match_mismatch_score,  # Match/Mismatch
                matrice_score[i - 1][j] + (cout_gap_ouverture if matrice_score[i - 1][j] == (i-1) * cout_gap_prolongement else cout_gap_prolongement),  # Gap dans seq2
                matrice_score[i][j - 1] + (cout_gap_ouverture if matrice_score[i][j - 1] == (j-1) * cout_gap_prolongement else cout_gap_prolongement)  # Gap dans seq1
            )

    # appel de la fonction de traceback
    alignement_seq1, alignement_seq2 = traceback_gap(matrice_score, seq1, seq2, matrice_distance, cout_gap_ouverture, cout_gap_prolongement)
    
    score_final = matrice_score[len_seq1][len_seq2]
    
    return alignement_seq1, alignement_seq2, score_final

# traceback adaptée
def traceback_gap(matrice_score, seq1, seq2, matrice_distance, cout_gap_ouverture, cout_gap_prolongement):
    alignement_seq1 = ""
    alignement_seq2 = ""
    i, j = len(seq1), len(seq2)

    while i > 0 or j > 0:
        if i > 0 and j > 0 and matrice_score[i][j] == matrice_score[i - 1][j - 1] + matrice_distance[alphabet.index(seq1[i - 1])][alphabet.index(seq2[j - 1])]:
            alignement_seq1 += seq1[i - 1]
            alignement_seq2 += seq2[j - 1]
            i -= 1
            j -= 1
        elif i > 0 and matrice_score[i][j] == matrice_score[i - 1][j] + (cout_gap_ouverture if matrice_score[i - 1][j] == (i-1) * cout_gap_prolongement else cout_gap_prolongement):
            alignement_seq1 += seq1[i - 1]
            alignement_seq2 += '-'  # Gap dans seq2
            i -= 1
        else:
            alignement_seq1 += '-'
            alignement_seq2 += seq2[j - 1]  # Gap dans seq1
            j -= 1

    alignement_seq1 = alignement_seq1[::-1]
    alignement_seq2 = alignement_seq2[::-1]

    return alignement_seq1, alignement_seq2

# choix des coûts
score_match = 1
score_mismatch = -1
cout_gap_ouverture = -2
cout_gap_prolongement = -1

# matrice de distance
matrice_distance = creer_matrice_distance(alphabet, score_match, score_mismatch)

# calcul de l'alignement
alignement_seq1, alignement_seq2, score_final = alignement_needleman_wunsch_gaps('CAT', 'TCT', matrice_distance, cout_gap_ouverture, cout_gap_prolongement)

print("Alignement:")
print(alignement_seq1)
print(alignement_seq2)
print(f"Score final: {score_final}")


Alignement:
-CAT
TC-T
Score final: 0.0


In [106]:
#Question 5: application sur les séquences vus en TD

#on pose les variables
alphabet = ['A', 'C', 'G', 'T']
score_match = 1
score_mismatch = -2
cout_gap = -1


matrice_distance = creer_matrice_distance(alphabet, score_match, score_mismatch)

# Séquences à aligner
seq1 = "CATGAC"
seq2 = "TCTGAAC"

# Calculer l'alignement
alignement_seq1, alignement_seq2, score_final = alignement_needleman_wunsch(seq1, seq2, matrice_distance, cout_gap)
print("Alignement Seq1 : ", alignement_seq1)
print("Alignement Seq2 : ", alignement_seq2)
print("Score final : ", score_final)




Alignement Seq1 :  -CATG-AC
Alignement Seq2 :  TC-TGAAC
Score final :  2.0


# 2. Petit exemple

In [108]:
#Question 1 

# Étape 1 on définit BLOSUM62 et l'alphabet
alphabet = "ACDEFGHIKLMNPQRSTVWY"  # liste des acides aminés
matrice_blosum62 = [

    [  4, -1, -2, -1, -2,  0, -2, -1, -1, -1, -1,  0, -1, -1, -1,  1,  0,  0, -3, -2],  #A
    [ -1,  9, -2, -2, -1, -1, -2, -1, -2,  0, -1, -1, -1, -1, -2,  0, -1,  0, -3, -2],  #C
    [ -2, -2,  6,  2, -3, -2, -1, -3, -1, -3, -2,  1,  0,  1,  1,  0, -1, -1, -4, -3],  #D
    [ -1, -2,  2,  5, -3, -2, -1, -3, -1, -3, -2,  0,0,  1,  0,  1,  0, -1, -4, -3],  # E
    [ -2, -1, -3, -3,  6, -2, -1,  0, -1,  2,  0, -2, -3, -3, -2, -1, -1,  0,  1, -2],  #F
    [  0, -1, -2, -2, -2,  6, -2, -1, -2, -2, -1, -1, 0, -1, -2,  0,  0, -1, -3, -2],  #G
    [ -2, -2, -1, -1, -1, -2,  8, -1,  0, -2, -1, -1, -1,  0,  1, -1, -1, -2, -3, -2],  #H
    [ -1, -1, -3, -3,  0, -1, -1, 5, -1,  2,  0, -1, -1, -1, -2, -1, -1,  0, -2, -2],  #I
    [ -1, -2, -1, -1, -1, -2,  0, -1,  5, -1, -1,  0, -1,  0,  1, -1, -1, -2, -3, -2],  #k
    [ -1,  0, -3, -3,  2, -2, -2,  2, -1,  5,  2, -1, -2, -1, -2, -1, -1, -1, -1, -1],  #L
    [ -1, -1, -2, -2,  0, -1, -1,  0, -1,  2,  5, -1, -2, -1, -2, -1, -1, -1, -1, -1],  #M
    [  0, -1,  1,  0, -2, -1,-1, -1,  0, -1, -1,  7,  0,  0,  0,  0, -1, -1, -4, -3],  #N
    [ -1, -1,  0,  0, -3,  0, -1, -1, -1, -2, -2,  0,  6,  1,  0,  0, -1, -1, -4, -3],  #P
    [ -1, -1,  1,  1, -3, -1,  0, -1,  0, -1, -1,  0,  1,  6,  1,  0, -1, -1, -4, -3],  #Q
    [ -1, -2,  1,  0, -2, -2,  1, -2,  1, -2, -2,  0,  0,  1,  6,  0, -1, -1, -4, -3],  #r
    [  1,  0,  0,  1, -1,  0,-1, -1, -1, -1, -1,  0,  0,  0,  0,  4,  0,  0, -3, -2],  #S
    [  0, -1, -1,  0,-1,  0, -1, -1, -1, -1, -1, -1, -1, -1, -1,  0,  5,  0, -3, -2],  #T
    [  0,  0, -1, -1,  0,-1, -2,  0, -2, -1, -1, -1, -1, -1, -1,  0,  0,  4, -3, -2],  #V
    [ -3, -3, -4, -4,  1, -3, -3, -2, -3, -1, -1, -4, -4, -4, -4, -3, -3, -3, -3, 11, 2],  #W
    [ -2, -2, -3, -3, -2, -2, -2, -2, -2, -1, -1, -3, -3, -3, -3, -2, -2, -2,  2,  9]   #y
]

# Étape 2 : on redéfinit les paramètres

seq1 = "MGPSENDPNLFVALYDFVASGDNTLSITKGEKLRVLGYNHNGEWCEAQTKNGQGWVPSNYITPVNSLEKHSWYHGPVSRNAAEYLLSSGINGSFLVRESESSPGQRSISLRYEGRVYHYRINTASDGKLYVSSESRFNTLAELVHHHSTVADGLITTLHYPAP"
seq2 = "GAMDPSEALQRPVASDFEPQGLSEAARWNSKENLLAGPSENDPNLFVALYDFVASGDNTLSITKGEKLRVLGYNHNGEWCEAQTKNGQGWVPSNYITPVNSLEKHSWYHGPVSRNAAEYLLSSGINGSFLVRESESSPGQRSISLRYEGRVYHYRINTASDGKLYVSSESRFNTLAELVHHHSTVADGLITTLHYPAPKRNKPTIYGVSPNYDKWEMERTDITMKHKLGGGQYGEVYEGVWKKYSLTVAVKTLKEDTMEVEEFLKEAAVMKEIKHPNLVQLLGVCTREPPFYIITEFMTYGNLLDYLRECNRQEVSAVVLLYMATQISSAMEYLEKKNFIHRNLAARNCLVGENHLVKVADFGLSRLMTGDTYTAHAGAKFPIKWTAPESLAYNKFSIKSDVWAFGVLLWEIATYGMSPYPGIDLSQVYELLEKDYRMERPEGCPEKVYELMRACWQWNPSDRPSFAEIHQAFETMFQESSISDEVEKELGKRGT"

#paramètres : 
cout_gap_ouverture = -11
cout_gap_prolongement = -1


#on aligne les séquences à l'aide de la fonction définit ci-dessus qui prend en compte différents couts suivant la nature du gap
alignement_seq1, alignement_seq2, score_final = alignement_needleman_wunsch_gaps(seq1, seq2, matrice_blosum62, cout_gap_ouverture, cout_gap_prolongement)


print(f"Alignement de séquence 1 : {alignement_seq1}")
print(f"Alignement de séquence 2 : {alignement_seq2}")
print(f"Score d'alignement final : {score_final}")


Alignement de séquence 1 : --M---------------------------------GPSENDPNLFVALYDFVASGDNTLSITKGEKLRVLGYNHNGE-WCEAQTKNGQG-WVPSNYITPVNSLEKHS-WYHGPVSRNAAEYLLSSGINGSFLVRESESSPGQRSISLRYEGRVYHYRINTASDGKLYVSSESRFNTLAELVHHHSTV-----------A-----------------------D-----------------G------------------------------------------L-----------I----T-------------------------T-----------------------------------------L--------H-----------------------------------------------------------Y--------P--------A---------P-------------------------------
Alignement de séquence 2 : GAMDPSEALQRPVASDFEPQGLSEAARWNSKENLLAGPSENDPNLFVALYDFVASGDNTLSITKGEKLRVLGYNHNGEW-CEAQTKNGQGW-VPSNYITPVNSLEKHSWY-HGPVSRNAAEYLLSSGINGSFLVRESESSPGQRSISLRYEGRVYHYRINTASDGKLYVSSESRFNTLAELVHHHSTVADGLITTLHYPAPKRNKPTIYGVSPNYDKWEMERTDITMKHKLGGGQYGEVYEGVWKKYSLTVAVKTLKEDTMEVEEFLKEAAVMKEIKHPNLVQLLGVCTREPPFYIITEFMTYGNLLDYLRECNRQEVSAVVLLYMATQISSAMEYLEKKNFIHRNLAARNCLVGENHLVKVADFGLSRLMTGDTYTAHAGAKFPIKWTAPESLAYNKFSIKSDVWAFGVLLWEIATYGMSPYPGIDLSQVYELLEKDYRMERPEGC

# 2. Smith-Waterman (Bonus)

In [120]:
#Question 1 :

# proposition d'algorithme de Smith-Waterman
def smith_waterman(seq1, seq2, score_match=2, cout_espace=-1, cout_mismatch=-1):
    longueur1, longueur2 = len(seq1), len(seq2)
    # Initialiser la matrice de scores avec des zéros
    matrice_score = np.zeros((longueur1 + 1, longueur2 + 1))
    score_maximal = 0 # (le score maximal donne la meilleure correspondance locale)
    position_maximale = None # (la position maximal servira de point de départ pour l'étape de traceback)

    # on remplit la matrice de score suivant cette fois la récurrence de Smith-Waterman
    for i in range(1, longueur1 + 1):
        for j in range(1, longueur2 + 1):
            correspondance = matrice_score[i-1, j-1] + (score_match if seq1[i-1] == seq2[j-1] else cout_mismatch)
            suppression = matrice_score[i-1, j] + cout_espace
            insertion = matrice_score[i, j-1] + cout_espace
            matrice_score[i, j] = max(0, correspondance, suppression, insertion)
            
            # suivi de la position et de la valeur maximum/maximal
            if matrice_score[i, j] >= score_maximal:
                score_maximal = matrice_score[i, j]
                position_maximale = (i, j)

    # comme pour Needlman : traceback pour trouver l'alignement local
    alignement1, alignement2 = '', ''
    i, j = position_maximale
    while matrice_score[i, j] != 0:
        if i > 0 and j > 0 and matrice_score[i, j] == matrice_score[i-1, j-1] + (score_match if seq1[i-1] == seq2[j-1] else cout_mismatch):
            alignement1 = seq1[i-1] + alignement1
            alignement2 = seq2[j-1] + alignement2
            i -= 1
            j -= 1
        elif i > 0 and matrice_score[i, j] == matrice_score[i-1, j] + cout_espace:
            alignement1 = seq1[i-1] + alignement1
            alignement2 = '-' + alignement2
            i -= 1
        else:
            alignement1 = '-' + alignement1
            alignement2 = seq2[j-1] + alignement2
            j -= 1

    # on peut essayter de localiser l'alignement dans les séquences d'origine
    debut_seq1, fin_seq1 = i, position_maximale[0] - 1
    debut_seq2, fin_seq2 = j, position_maximale[1] - 1

    return {
        'score': score_maximal,
        'alignement_sequence1': alignement1,
        'alignement_sequence2': alignement2,
        'debut_sequence1': debut_seq1,
        'fin_sequence1': fin_seq1,
        'debut_sequence2': debut_seq2,
        'fin_sequence2': fin_seq2
    }


In [127]:
# Question 2
seq1 = "GCACTAGACC"
seq2 = "GCATCGAC"

# avec Smith-Waterman
print("Suivant l'algorithme de Smith-Waterman, on obtient : ")
resultat = smith_waterman(seq1, seq2)

print("Score:", resultat['score'])
print("Alignement de la séquence 1:", resultat['alignement_sequence1'])
print("Alignement de la séquence 2:", resultat['alignement_sequence2'])
print("Position dans la séquence 1:", resultat['debut_sequence1'], "-", resultat['fin_sequence1'])
print("Position dans la séquence 2:", resultat['debut_sequence2'], "-", resultat['fin_sequence2'])

# avec Needlman-Wunsch

print("--------------------------------------------------------")

print("Suivant l'algorithme de Needlman-Wunsch , on obtient : ")
# on choisit des paramètres semblables à l'algorithme de Smith-Waterman
alphabet = ["A", "C", "G", "T"]
score_match = 2
score_mismatch = -1
cout_gap = -1

# on fait la matrice de distance
matrice_distance = creer_matrice_distance(alphabet, score_match, score_mismatch)

# on procède au calcul de l'alignement
alignement_seq1, alignement_seq2, score_final = alignement_needleman_wunsch(seq1, seq2, matrice_distance, cout_gap)
print("Alignement Seq1 : ", alignement_seq1)
print("Alignement Seq2 : ", alignement_seq2)
print("Score final : ", score_final)



Suivant l'algorithme de Smith-Waterman, on obtient : 
Score: 12.0
Alignement de la séquence 1: GCACTAGAC
Alignement de la séquence 2: GCA-TCGAC
Position dans la séquence 1: 0 - 8
Position dans la séquence 2: 0 - 7
--------------------------------------------------------
Suivant l'algorithme de Needlman-Wunsch , on obtient : 
Alignement Seq1 :  GCACTAGACC
Alignement Seq2 :  GCA-TCGA-C
Score final :  11.0
